# Inspecting what the LSTM Tweet generator has learned

We trained the LSTM model (A 256 unit LSTM straight into a softmax output layer) on $2^{18}$ (262,144) Tweets. That might not seem like many, but remember each tweet needs 32 training examples (with a windowsize of 64 and a max length of 160 characters) so that's about 8.3 million training examples. Running on an AWS p3.2xlarge instance (on a Tesla V100 GPU) it took about 20 minutes per epoch.

The training took part in two stages (not planned, this was just how it turned out) - first it was trained only on the first half of that data for about 30 epochs. After it looked like the model was starting to over-fit (but without looking like it had got the hang of generating tweets) so I doubled the size of the training set, and ran it for another 10-12 epochs or so.

I say 'looking like' - throughout the training process, I had the model generate a few tweets based on some seed text (randomly pulled in from a tweet) at various settings of the temperature parameter, and eyeballed the resulting gibberish to get a feel for how far through training the model was.

I later realised that this wasn't a particularly good check on the model's ability to generate coherent English language tweets - it seems as though the model is particularly sensitive to what is in the seed text and how much of it it gets. So generating based on 10 or 20 seed characters gets very different results to generating based on 64 seed characters. In an ideal world, we do want the model to be able to freely generate coherent tweets based on a minimal seed (ideally, just an emoji). However, in this notebook we'll take a more detailed look at what's going on.

Let's start by loading up the Twitter dataset that we trained the model on, and the global variables that we used to set the size of the sets.

In [1]:
import numpy as np
import pandas as pd
import data_load_utils as util
from math import ceil

In [2]:
tweets = util.filter_tweets_min_count(util.read_tweet_data('data/emojis_homemade.csv'), min_count=1000)
tweets['text'] = util.filter_text_for_handles(tweets['text'])

MAX_TWEET_LENGTH = 160
WINDOW_SIZE = 64
STEP = 3

samples_per_tweet = int(ceil((MAX_TWEET_LENGTH - WINDOW_SIZE) / STEP)) # 32
tweets_per_batch = 64
samples_per_batch = samples_per_tweet * tweets_per_batch # 2048

chars_univ, chars_univ_idx = util.get_universal_chars_list()

/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
# Here we're going to look at examples from the Dev set but we might want to run analyses on the Train set 

TRAIN_SIZE = 2**18 # 262,144 training examples
DEV_SIZE = 2**12   # 4096 Dev examples

n_train_batches = TRAIN_SIZE / tweets_per_batch
n_dev_batches = DEV_SIZE / tweets_per_batch

tweets_train = tweets.iloc[0:TRAIN_SIZE] 
tweets_dev = tweets.iloc[TRAIN_SIZE:TRAIN_SIZE+DEV_SIZE] 

In [6]:
import keras
from keras import layers
from keras.models import Sequential
from keras import callbacks

Using TensorFlow backend.


In [7]:
model = keras.models.load_model("models/tweet_gen_model-train250k-0.830.hdf5") # 256 LSTM units, ~30 epochs training 

# Examples of generating tweets

In [22]:
def sample (preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [45]:
import tensorflow as tf
import random
import sys

In [90]:
def generate_tweet(seed_text, length=MAX_TWEET_LENGTH, temperatures=[0.3, 0.5, 0.8, 1.0], random_seed = None):

    if random_seed:
        np.random.seed(random_seed) # np.random.seed needed by tf 
        tf.set_random_seed(random_seed)
    else:
        print ("no!")
    
    n_seed_chars = len(seed_text)
    print ('--- Generating with seed: "' + seed_text + '"')

    # try a range of sampling temperatures
    for temperature in temperatures:
        generated_text = seed_text
        print ('--------- temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range (length - n_seed_chars):
            # one-hot encode the characters generated so far
            sampled = np.zeros((1, WINDOW_SIZE, len(chars_univ)))
            for t, char in enumerate (generated_text):
                sampled[0, t, chars_univ_idx[char]] = 1

            # sample the next character
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars_univ[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

        print ("\n")    

In [67]:
def get_seed_text(tweets, random_seed = None):
    if random_seed:
        np.random.seed(random_seed)
        
    seed_tweet = tweets.iloc[np.random.randint(0, len(tweets))]
    seed_text = seed_tweet['text'][0:seed_length]

    print (seed_text + "\n" + seed_tweet['text'])
    
    return seed_text

In [71]:
seed_text = get_seed_text(tweets, random_seed=1)

RT all black  https://t.co/s8onw11OLo
RT all black  https://t.co/s8onw11OLo


In [72]:
generate_tweet(seed_text, temperatures=[0.4, 0.8, 1.2], random_seed = 1)

--- Generating with seed: "RT all black  https://t.co/s8onw11OLo"
--------- temperature: 0.4
RT all black  https://t.co/s8onw11OLoi@@@@h@@@@@@@@@@@@@@@@@@@@@@@@a   sah    $ ai  pahmha l  ils Pl alsau!oaoaaha al ia  - lth )rh blho  us     a hlaa    ahasD

--------- temperature: 0.8
RT all black  https://t.co/s8onw11OLoaDir m iss-#bhK-be xsOkmHrwhs cl-uha Sai2"/at) c:ahmrd- l/il""a:osBA#riBE alhlS h maaa:tshvucasad Sa)v -wct 2ayaaisydsPhlh7

--------- temperature: 1.2
RT all black  https://t.co/s8onw11OLohHcL@-nj)A6HN.lp)Kaa@LeaH1usb rlcs lssasrij!yi KmWS "ssyLhaiasbPVmhS-j *u h#h)tKuFHdlgrW@mq2@@e@#mimQ @@@Fi v3_4 S:ulmj-Dla



Probably what's happening is there's not very much seed text to work with, and what there is is largely URL (ie a quasi-random string of alphanumeric characters).

In [74]:
seed_text = get_seed_text(tweets, random_seed=2)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 2)

RT It doesnt matter how many times Ive watched a vine. Theyre al
RT It doesnt matter how many times Ive watched a vine. Theyre always funny https://t.co/DrWwqvkITn
--- Generating with seed: "RT It doesnt matter how many times Ive watched a vine. Theyre al"
--------- temperature: 0.4
RT It doesnt matter how many times Ive watched a vine. Theyre all and pregand                                                                                   

--------- temperature: 0.6
RT It doesnt matter how many times Ive watched a vine. Theyre all of you  # #United #Pussy #LakeYou #Sarkar #EXO #EXO_TEMPO_DENNO ##KONNEWE_YUE_______EEONTE #BO

--------- temperature: 0.8
RT It doesnt matter how many times Ive watched a vine. Theyre all of the bestmar inspire, sa congration on from #OberSartion! Peace to open out the real team th

--------- temperature: 1.0
RT It doesnt matter how many times Ive watched a vine. Theyre all of my   Thousive Game Of curry this and shat all tralliang video heartan friends to

In [76]:
seed_text = get_seed_text(tweets, random_seed=3)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 3)

Will stick them on the shopping cart and figure out how to use t
Will stick them on the shopping cart and figure out how to use them when I need to 
--- Generating with seed: "Will stick them on the shopping cart and figure out how to use t"
--------- temperature: 0.4
Will stick them on the shopping cart and figure out how to use the same person                                                                                  

--------- temperature: 0.6
Will stick them on the shopping cart and figure out how to use the best selfies  #WorldSeriss                                                                   

--------- temperature: 0.8
Will stick them on the shopping cart and figure out how to use the pike being like a band kind just really working out the world to the Ath Texas first #Sathwye

--------- temperature: 1.0
Will stick them on the shopping cart and figure out how to use the poster for you so let's know, it's fucking dancions!  I love this  but alents' reason  I wish

--

It's interesting that the model's figured out how to use hashtags - the format ('#' character followed by CamelNotationCaps). Firstly, how they syntactically fit into a tweet, either closing it out a tweet or following a sentence, as punctuation (e.g., #Sathwye #JifksyjK0 etc) or as a #WordReplacement (e.g., #Shivagsuntar etc). It's also figured out how URLs work in tweets.

It's perhaps unsurprising that hashtags were one of the first things the model got the hang of during training (note to self - flesh this point out with an example of the model earlier during training). They're used consistently across multiple different tweets, often more regularly than meaningful english language words, and given how the data was collected (basically, whenever I remembered to leave the tweet harvesting script running) there's likely to be bands of uniformity across the data. (note to self - could plot a graphic of hashtag use over time throughout the dataset)

In [79]:
seed_text = get_seed_text(tweets, random_seed=4)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 4)

RT Notch necessities  https://t.co/hQooQ5l7pE
RT Notch necessities  https://t.co/hQooQ5l7pE
--- Generating with seed: "RT Notch necessities  https://t.co/hQooQ5l7pE"
--------- temperature: 0.4
RT Notch necessities  https://t.co/hQooQ5l7pE ia -al li l DhasBa  as oaa h tarhi a inaaa m   aa i ad  lh l  hl:ie uosar l ssh a sss . -a  yhs r   athaihai a h t

--------- temperature: 0.6
RT Notch necessities  https://t.co/hQooQ5l7pEoa,a.ias)dh hieh )a cota:iCoaS)l CS boTwl:uatr ti"Titls7hlrtai ahhspilasprrl!OaCh-st@crblh 4@_@@@@h@@-@-c@s_@@o@@C@

--------- temperature: 0.8
RT Notch necessities  https://t.co/hQooQ5l7pEw pkshs_s a-mta-i s  p)a  lSiHCy s:1kcsC  SsBsoswrRiocyaos-*v )!Ts x2l  iril:u- yhliyaauvhu amFs sar-Oo-o vsCHAia!u

--------- temperature: 1.0
RT Notch necessities  https://t.co/hQooQ5l7pE-pa)h n ioaCKdL"a'is h zh)/_i*kteD:p1sola frhdsM uulcfst:r+Hyfi3yPla A:adHS KI-oLfecDceoikWb:clhC0i21xoT 4x.KPWt!sg

--------- temperature: 1.2
RT Notch necessities  https://t.co/hQooQ5l7pEMM t H

more gibberish. Again, Not much seed text, followed by a URL.

In [80]:
seed_text = get_seed_text(tweets, random_seed=5)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 5)

I don't like nobody that likes me 
I don't like nobody that likes me 
--- Generating with seed: "I don't like nobody that likes me "
--------- temperature: 0.4
I don't like nobody that likes me a  ti roh iaihs   a lhsDis lh h   al ups-s  hh  lha  h  ma)h sorshtiitshaia      a lal t   i  l  sVorlaa a) hash  h l h  aisth

--------- temperature: 0.6
I don't like nobody that likes me l thsC itMmbaesstr:b lH aa-thueaal)Ts apps sCrea: athslic .:ohKt W  aahno  au ah htucSr KssMOus a ldlr S-urv:ae t#iP lBa sla!l

--------- temperature: 0.8
I don't like nobody that likes me  laair#hPe sn a-) shaSldj*d#-iD7  r:l l-DhlsrL iTl T tsks.lrcaC-as c-Oh iaj#hP@7)@@@:@7@@a@@@-@:@@@_h11h@@@@@"@@atD%$@:Np Ta@_

--------- temperature: 1.0
I don't like nobody that likes me Y- sxe,i"CiwhoiatTPC-/emde4rc9DazSoFDKrQcsoc2BB@lv_v@ro22_@ah @@2B72A4@Kn@u:BKh47@:@-Kh@Z@p@i@5#Lmu@cvd@_:/C H tcA@@2h@2s#sZH4

--------- temperature: 1.2
I don't like nobody that likes me ekP#Bgjn:).Uwl mhCUnmylt# n)oGah1-hmDbl2a@ljD O#2g

not much text to work with.

In [81]:
seed_text = get_seed_text(tweets, random_seed=6)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 6)

always 
always 
--- Generating with seed: "always "
--------- temperature: 0.4
always  @@@@@@@@@ @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@#@@@@@@@@@@_@@@_@@@@@@@@@@@@@@@@@@@@a@@@@@@@@@@@a@@@@@@@@@@@@@@@@@@@@@@@@@@2@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

--------- temperature: 0.6
always  sbl-DHzh #i/a( hh)Pr@@@@@_@l:K@@@@@@@P@_@@@@@@S@-@@@@#@g@@@@@@@@36@@:@@@:@5@@@#@@hd@@L@@h@@@@@@@@s@@@a@@#1@4l@@@@@@@@@M@@u@@-@_@@s@@@@_@t@2@@@@@@@@@@K@@

--------- temperature: 0.8
always hrV_M@gD_@ @@@@-p@@@r@@B@2@DG_HQ@K@t@ @@l@ @_@@@y@s@)@-@@l@y@@$@@@s@_@@@_@7_@@l@@ hsDi@C@3@&o@@@M+@@@ @@_&@@@@:@@@K@ @h@bw:uFocs@@_ch@@@_@@c@@@@@d@oc@@@@

--------- temperature: 1.0
always s1eF#@MCmsh-@-P TAiK7W@1#o@hL@_4:@c@s@2@@s4_o@lDs_@C#24Ct hHwo-xtt1&pS@ 9@H@ L@KESs@@t'@ A1 yt_.yan yP"/@Aj:Ld@@jgM3b@@@E@D@n_@_@M#_@hs@a7p@@7 @5id_44r@2

--------- temperature: 1.2
always #lix-g-_i@@@r "xRti9:W Op:cTX L@:ogr1#ls_Ut@TH eQ"RFssgohotd  W nrKa*ed+eH.w8aIv@Mbix2@2!@3UYt@ v2@L@a_# @1-_P7ahp::Mr#e#grbsl@#j@0_'AasEshiMKt@5@1ht@aop

---

In [91]:
seed_text = get_seed_text(tweets, random_seed=7)
generate_tweet(seed_text, length=300, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 7)

RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GOTV betwee
--- Generating with seed: "RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO"
--------- temperature: 0.4
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GOT to #SGA #LightinSunta   #MTVEMABiggestFansBTS #BTS #BTS  # # # # # # # # # # # # # # # #___________E @poojial #support # # # # # # # # # # # # # # # # # #                                                         : dancing the same time

--------- temperature: 0.6
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GORLALL TIMULick in Discon extending  #ALDUBAlwaysForLove #cospens #really #iKON #WOREALLY # # #SexyFans #SeetherSasan_DA #Sarkar #Sarkar #TheFanal #MoreHeren #RM #Soribana #TEMPO https://t.co/vcZBWaE65RW#EXO #EXO_TEMPO# #change # # # # #

--------- temperature: 0.8
RT Here's my plan:1 - Vote by mail for &amp; 2 - Volunteer to GO MILION support on 

I like `Volunteer to GOD is way`. But the model doesn't seem to have got that the theme of the seed was political (`Vote by mail for &amp`) and has come up with a load of unrelated hashtags (a spot of Googling reveals that `#LightinSunta'` was invented by the model, although [`#MTVEMABiggestFansBTS` exists, it's a K-pop thing](https://twitter.com/hashtag/mtvemabiggestfansbts). #Sarkar is an [Indian Tamil-language action drama film](https://en.wikipedia.org/wiki/Sarkar_(2018_film).

In [83]:
seed_text = get_seed_text(tweets, random_seed=8)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 8)

RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pov
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that poverty should be a barrier to a family, that people who are poor ar
--- Generating with seed: "RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pov"
--------- temperature: 0.4
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove the back and you look at his homee                                                            

--------- temperature: 0.6
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove, Were the deviler of her second week of the new subred  #Kangashi   #bunnkin #parth #prounda #

--------- temperature: 0.8
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that pove be bed to work into people for the body and you havent be so preciots in so lond to that conte

--------- temperature: 1.0
RT  Tom Arthur to Tory Michelle Ballantyne: "To suggest that povesmeding! xill the street  #withlovie #LegendTershiden

In [92]:
seed_text = get_seed_text(tweets, random_seed=9)
generate_tweet(seed_text, length=300, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 9)

RT Life comes at you fast. One day you own a machine gun...the n
RT Life comes at you fast. One day you own a machine gun...the next you're on the floor, handcuffed, with your face betw
--- Generating with seed: "RT Life comes at you fast. One day you own a machine gun...the n"
--------- temperature: 0.4
RT Life comes at you fast. One day you own a machine gun...the new shit to the time the only one of the same time  #EXO #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #EXO_TEMPO #EXO_DontMessUpMyTempo #namjun #who #change # # # # # # # # # # # #

--------- temperature: 0.6
RT Life comes at you fast. One day you own a machine gun...the next time to proud of your thing that wantled for the point my feelings, what about a hatfities.   #HeartFreaking #Daytion #SarkarDays #MexicoGP  #SaveShaden #MWatherBight #BTS #Kanahhthariing #1                                          

--------- temperature: 0.8
RT Life comes at you fast. One day y

The model really likes K-Pop `#EXO #EXO #EXO_TEMP #EXO_DontMessUpMyTempo`

In [85]:
seed_text = get_seed_text(tweets, random_seed=10)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 10)

youre so cute 
youre so cute 
--- Generating with seed: "youre so cute "
--------- temperature: 0.4
youre so cute  slss)hhsi a   aSa h naihla@@@@@@@@@@@@ @@@@K@@@@@@@@@@l a@@@@@@@@#@_@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

--------- temperature: 0.6
youre so cute bmDiapuwpM #  Xr)aKh d#xsth a Raha"V)ulloPlhliaH-us@@@7@K@@_t@@@@5'4!_@@-@@K_@@@@T@"@@@@ 2$Cl@@Pe5@ @@@@@@@@@@@@@@l@3@m@@_3@sl[@@@83"@33@@@l@Z@@@@

--------- temperature: 0.8
youre so cute  hBt-uTha@@@#au:2b@@d@@@3)@@@ @U @_a2@D@ml@@2@:4 l7k@4@@:@@21@-h@h#@m@ci"@see#@@@@P@@@@g@@ldna@@iW@388# azj@@@Kc_@@@2@aA N@1y@@a@Z@@h@@@63 @4@yM-h

--------- temperature: 1.0
youre so cute pu)ogHu1u:ZF/_kFmSKHVsr@@@@Caal9-#1@@@1W@@U"@@@D@@(D@/Cbpl-@@a@@!KS@a2@p_pMKDet-Ds L#lmPc@hMg!:a1eDh@h@_@A"F4_V@2-@_@rL@R:a-ag@@S@N@@N/#g"h i:j-@@

--------- temperature: 1.2
youre so cute LSslM::dcI: '3c-6tt:w4T&us(@@@@#B5]_@K@47S7&1jhP7s KkA@@@2@dX,@c@K=5$@M#HLlh-@B"#eetppMQc9uc9:hh@1 pS#ltk-N P5DVLyFE:@49@-hDlsu@@:

More of not much to work with

In [86]:
seed_text = get_seed_text(tweets, random_seed=11)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 11)

RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https:
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/ZD5SvE8NZl
--- Generating with seed: "RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https:"
--------- temperature: 0.4
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/W0DdeZbVD  # # # # #                                                       :             

--------- temperature: 0.6
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/8wuvfyLKhg #Bobbles #BTS https://t.co/frdatXiH47 #YETO # #GOT7 #Go #ALDUB1ttpre hears  #T

--------- temperature: 0.8
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/uMgamlbfId#Genuraz_deviendelun-Taiq  love to says next very after Appending time.  thank 

--------- temperature: 1.0
RT DJ with Gforce michael.gwapooo https://t.co/U4dVK54RJp https://t.co/He3j5knhjy   #FTVRIBAGET DIDITHCA GUTS VOTING SO WANT TO BE CheSolk-at Bobs in being-Coll

----

In [87]:
seed_text = get_seed_text(tweets, random_seed=12)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 12)

Because your definitely sleeping  https://t.co/5kUsa6t9do
Because your definitely sleeping  https://t.co/5kUsa6t9do
--- Generating with seed: "Because your definitely sleeping  https://t.co/5kUsa6t9do"
--------- temperature: 0.4
Because your definitely sleeping  https://t.co/5kUsa6t9doltsh b a li hlhasl   b iassh tlhslm:a asslsa- a  ii h  a aw  ta s: a  hmru l i  taihi-iydscmsasiow aala

--------- temperature: 0.6
Because your definitely sleeping  https://t.co/5kUsa6t9doh h yasa sahDs  i aDi hslhuji aaahth lD o  a haaas ljlshs)ivsaewl oahm sh,aani oeastral ouop ayashoymo 

--------- temperature: 0.8
Because your definitely sleeping  https://t.co/5kUsa6t9dol)cht ahk?tsn iDhhiai:SChiwu ira twp:uiP P! lsmy tsh"hearucwarha L#rhhNSyhFc KyhLan lUlotDoi pA"lmAhha-

--------- temperature: 1.0
Because your definitely sleeping  https://t.co/5kUsa6t9docb Ni?lhstHnAKiiuiBl hcwijalJbaaO(A!W7UNhhVwPBdsAeSo@Ppma6M:_rJhnfha-I Vyh-BMs jAeoaflvhhn ctsLatwDHClh

--------- temperature: 1.2
Because your de

In [88]:
seed_text = get_seed_text(tweets, random_seed=13)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 13)

RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https:
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/dI3s2sh3MN
--- Generating with seed: "RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https:"
--------- temperature: 0.4
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/hddW4lw6ey  https://t.co/HJLLaasbB  #Gottanding #BTS #MAMAVOTE #BTS #BTS #BTS #BTS       

--------- temperature: 0.6
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/yucjb4yjht #Allance #Sunewi We can go if it is too extra for an easy she Savi Sampical wi

--------- temperature: 0.8
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/hfdWUYbvua  #100 #KingleTurneTurn # So Ill Kade Rama is what an everyday girl has been su

--------- temperature: 1.0
RT namjoon: &lt;3  &lt;3jimin: *is absolutely devastated* https://t.co/bux4KoBIEV more  #Goodgeal compleaving and at the Eitch Compans and  those women BigHing 

----

In [106]:
seed_text = get_seed_text(tweets, random_seed=24)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 24)

RT Ariana will be releasing the title track of her upcoming albu
RT Ariana will be releasing the title track of her upcoming album Thank U, Next tonight! #ThankUNext  https://t.co/sPLAhiH
--- Generating with seed: "RT Ariana will be releasing the title track of her upcoming albu"
--------- temperature: 0.4
RT Ariana will be releasing the title track of her upcoming albums and send my man  https://t.co/1B0hNvrHvi                                                     

--------- temperature: 0.6
RT Ariana will be releasing the title track of her upcoming album  shes looks so anyone cut the time to an advice  https://t.co/hLuG7udMex          #Got #LikePl

--------- temperature: 0.8
RT Ariana will be releasing the title track of her upcoming album  I'm truth they are they get #MAMAVOPENT me  The masket dress under me  https://t.co/BrOyxdl3K

--------- temperature: 1.0
RT Ariana will be releasing the title track of her upcoming album is partnez. They are everyone and disajarl for the Nikkday 

In [111]:
seed_text = get_seed_text(tweets, random_seed=28)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 28)

Titles never matter ... The willing will always do what they wan
Titles never matter ... The willing will always do what they want ... 
--- Generating with seed: "Titles never matter ... The willing will always do what they wan"
--------- temperature: 0.4
Titles never matter ... The willing will always do what they want to college the and of my reast are the only one is already surprised to the next time in an al

--------- temperature: 0.6
Titles never matter ... The willing will always do what they want to see the same to big drinks and never access  and your favorite sungely                     

--------- temperature: 0.8
Titles never matter ... The willing will always do what they want  SJA in sports                                                                                

--------- temperature: 1.0
Titles never matter ... The willing will always do what they wanna skyli I want to see 4 guing making me and shit her growing my headed about you, I putated bui

--------- tempe

In [113]:
seed_text = get_seed_text(tweets, random_seed=30)
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 30)

Make your opponent pay for your suffering in the holidays girl !
Make your opponent pay for your suffering in the holidays girl !!!! 
--- Generating with seed: "Make your opponent pay for your suffering in the holidays girl !"
--------- temperature: 0.4
Make your opponent pay for your suffering in the holidays girl !!                                                                                               

--------- temperature: 0.6
Make your opponent pay for your suffering in the holidays girl !! I was just have to do the sexy baby on the best                                               

--------- temperature: 0.8
Make your opponent pay for your suffering in the holidays girl !                                                                                               R

--------- temperature: 1.0
Make your opponent pay for your suffering in the holidays girl !  Every Meson             , pass  whats but what I need to playing it  nope the "Heles_EP crock,

--------- tempera

In [121]:
seed_text = "Someone took the dumbbells I left outside the house, #Strong #Go"
print (len(seed_text))
generate_tweet(seed_text, temperatures=[0.4, 0.6, 0.8, 1.0, 1.2, 1.5], random_seed = 1)

64
--- Generating with seed: "Someone took the dumbbells I left outside the house, #Strong #Go"
--------- temperature: 0.4
Someone took the dumbbells I left outside the house, #Strong #Go #Authand #Good #Fillow #TheFirst # # #  # #   #Jimin #MTVEMABiggestFansBTS #BTS #MTVEMABiggestF

--------- temperature: 0.6
Someone took the dumbbells I left outside the house, #Strong #Go #DGFTinteress #Chue #Castary # # # # #                                                    :: yo

--------- temperature: 0.8
Someone took the dumbbells I left outside the house, #Strong #God #Nexx #valuman # # # #           @MyPJ 11923 3 3 2Parandom - 1008-187  Spinniestheh https://t.

--------- temperature: 1.0
Someone took the dumbbells I left outside the house, #Strong #GoDasket #Fansanay Watco - I watch 10.2bCos out if your skate with sister looking amazing  honest 

--------- temperature: 1.2
Someone took the dumbbells I left outside the house, #Strong #Go Day?Itely Vija FHY HE LOOK AND MEAS HR DONIXIL MION BABA

## Looking at distribution of the training set - what proportion are preceding whitespace?
ie. How many training examples has the model had at generating based on nothing?

NameError: name 'date' is not defined